In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

def dydx(x,y):
    return x**3-y/x

def sigmoid(z):
    return 1/(1+math.exp(-z))

def N(x,p,n):
    result=0
    for i in range(n):
        result+=p[n+i]*sigmoid([p[i]*x-p[2*n+i]])
    return result

def precisesolve(x):# 精确解
    return x**4/5+1/(5*x)

def yt(x,p):
    return 0.4-N(1,p,np.size(p)/3)+x*N(x,p,np.size(p)/3)

def asgradyt(x,p,epsilon):
    return (yt(x+epsilon,p)-yt(x,p))/epsilon

def targetfunction(xs,p):
    for xi in xs:
        return (asgradyt(xi,p,1e-6)-dydx(xi,yt(xi,p)))**2
    
def gradtargetfunction(xs,p,epsilon):
    result=[]
    ptemp=p.copy()
    for i in range(np.size(p)):
        ptemp[i]+=epsilon
        result.append((targetfunction(xs,ptemp)-targetfunction(xs,p))/epsilon)
        ptemp=p.copy()
    return np.mat(result).T

def asnewton(xs,p,epsilon,method):#对p搜索
    H=np.eye(np.size(p))
    time=0
    grad=gradtargetfunction(xs,p,1e-6)
    while True:
        if(np.linalg.norm(gradtargetfunction(xs,p,1e-6))<=epsilon):
            break
        dk=-H*grad
        
        alpha = 1#ajmijo没改
        #armijo
        while True:
            if targetfunction(xs,p+alpha*dk)<=targetfunction(xs,p)+0.5*alpha*(grad.T*dk):
                break
            alpha=alpha*0.5

        sk=alpha*dk
        gk=grad
        grad=gradtargetfunction(xs,p,1e-6)
        yk=grad-gk
        if method=='BFGS':
            H=H+(1+(yk*yk.T)/(yk.T*sk)*(sk*sk.T)/(yk.T*sk)-(sk*yk.T*H+H*yk*sk.T)/(yk.T*sk))
        if method=='DFP':
            H=H+(sk*sk.T)/(sk.T*yk)-(H*yk*yk.T*H)/(yk.T*H*yk)
        time+=1
    return p

p=[1,1,1,1,1,1,1,1,1]
x=np.linspace(1,2,100)
pBFGS=asnewton(x,p,1e-6,'BFGS')
pDFP=asnewton(x,p,1e-6,'DFP')

yBFGS=[yt(xi,pBFGS) for xi in x]
yDFP=[yt(xi,pDFP) for xi in x]
yP=[precisesolve(x) for xi in x]

plt.plot(x,yBFGS)
plt.plot(x,yDFP)
plt.plot(x,yP)
plt.title("neuron network")
plt.legend('BFGS','DFP','precise')
plt.show()

TypeError: 'float' object cannot be interpreted as an integer